In [1]:
import ee
import geemap
import math

In [4]:
geemap.set_proxy(port=8800)
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<h1>构建Sentinel影像集并进行特征采集

In [17]:
# ——用户自定义函数——#
def reScale(image):
    return image.divide(10000)

def rmCloudByQA60(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask  = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    mask2 = image.select("B2").lt(2000)
    return image.updateMask(mask).updateMask(mask2)

def getS2ImageCol(roi, s2, dateRange):
    return (ee.ImageCollection(s2)
            .filterBounds(roi)
            .filterDate(dateRange)
            .map(rmCloudByQA60)
            .select([
                'B1','B11','B12','B2','B3','B4','B5',
                'B6','B7','B8','B8A','B9'
            ])
            .map(reScale)
           )

# ——准备Sentinel影像——#
start = '2019-01-01'
end = '2019-12-31'
roi = (ee.FeatureCollection(
         "projects/ee-wl1945790713/assets/bareland_extraction/focal_img_500_2_shape_Asia"
       )
       .geometry()
       .dissolve()
       .simplify(10000)
     )
dateRange = ee.DateRange(start, end)
s2 = "COPERNICUS/S2_SR_HARMONIZED"
s2Col = getS2ImageCol(roi, s2, dateRange)
s2Image = s2Col.median().clip(roi)

# 可视化真彩色
Map.addLayer(
    s2Image,
    {'bands': ['B4','B3','B2'], 'min': 0, 'max': 0.3},
    'Sentinel-2 RGB'
)

# 计算各种指数并添加波段
indices = {
    'MNDWI': ['B3','B11'],
    'NDBI' : ['B11','B8'],
    'NDVI' : ['B8','B4'],
    'NDSI' : ['B4','B1'],
    'NDSDI':['B4','B12'],
    'NDSAI':['B11','B4'],
}
for name, bands in indices.items():
    img_idx = s2Image.normalizedDifference(bands).rename(name)
    s2Image = s2Image.addBands(img_idx)
GI = s2Image.expression(
    "0.3 * R + 0.59 * G + 0.11 * B",
    {
        'R': s2Image.select('B4'),
        'G': s2Image.select('B3'),
        'B': s2Image.select('B2')
    }
).rename("GI")

#——添加其他影像数据集——#
dem = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
slope = ee.Terrain.slope(dem).rename("slope")
aspect = ee.Terrain.aspect(dem).rename("aspect")

In [14]:
#Map.addLayer(GI, {'min': 0, 'max': 6}, 'GI')
#Map.addLayer(roi, {'color': 'red'}, 'ROI')
Map

Map(bottom=25159.0, center=[39.7958755252971, 83.47412109375001], controls=(WidgetControl(options=['position',…

In [ ]:
glcm = GI.select('GI').toByte().glcmTexture({size: 3, scale: 1}
)